# Importing Libraries

In [ ]:
import ee
import geemap
import xarray as xr

In [ ]:
!pip install xee
import xee

In [ ]:
ee.Authenticate()
ee.Initialize(project = '-----Insert ProjectID Here-----', opt_url='https://earthengine-highvolume.googleapis.com')

# Area of Interest

In [ ]:
data = ee.FeatureCollection('FAO/GAUL/2015/level2')
aoi = data.filter(ee.Filter.eq('ADM0_NAME', 'Kenya').eq('ADM2_NAME', 'Uasin Gishu'))
aoi = aoi.geometry()


In [ ]:
Map = geemap.Map(height = 500)
Map.add("basemap_selector")
Map.addLayer(aoi, {}, 'Uasin Gishu County')
Map.centerObject(aoi, 9)
Map

# Landsat 8/9 Level 2 Collection 2 Tier 1 Imagery

In [ ]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").merge(ee.ImageCollection("LANDSAT/LC09/C02/T1_L2"))\
          .filterDate('2019-01-01', '2019-03-31').filterBounds(aoi).filter(ee.Filter.lt('CLOUD_COVER', 10)).sort('system:time_start')

In [ ]:
landsat

# Generating an NDVI Mask Function


In [ ]:
def ndvi(img):
  band = img.select('SR.*').multiply(2.75e-05).add(-0.2)
  index = band.normalizedDifference(['SR_B5','SR_B4']).rename('ndvi')
  return index.copyProperties(img, ['system:time_start'])

# Masking Landsat Imagery with an NDVI Mask

In [ ]:
landsat_ndvi = landsat.map(ndvi)
landsat_ndvi

# Generating Images Pre and Post Fires in Uasin Gishu County in 2019

In [ ]:
ndvi_before = landsat_ndvi.filterDate('2019-01-01','2019-01-31').median().rename('ndvi_before')
ndvi_after = landsat_ndvi.filterDate('2019-02-22','2019-03-21').median().rename('ndvi_after')

# Generating NDVI Change Between the Before and After Fire Images in Uasin Gishu

In [ ]:
ndvi_change = ndvi_before.subtract(ndvi_after).rename('ndvi_change')

# Stacking the Before, After and Change Images of Uasin Gishu as Bands in a Single Image

In [ ]:
ndvi_stack = ee.Image.cat([ndvi_before, ndvi_after, ndvi_change])
ndvi_stack

# Image Segementation

In [ ]:
segmentation = ee.Algorithms.Image.Segmentation.SNIC(ndvi_stack).select('.*mean')
segmentation

# Computing Amount of Lost Area Due to Fire in Square Kilometers

In [ ]:
def burned_area(img):
  img_thr = img.gt(0.15).selfMask()
  thr_pix = img_thr.multiply(ee.Image.pixelArea().divide(1e6))
  return thr_pix

In [ ]:
forest_lost = burned_area(segmentation.select('ndvi_change_mean'))

In [ ]:
forest_lost.reduceRegion(reducer = ee.Reducer.sum(), geometry = aoi, scale = 30).values().get(0)

# Converting Stacked NDVI Image to an XArray Dataset

In [ ]:
ds  = xr.open_dataset(segmentation, engine = 'ee', crs = 'EPSG:4326', geometry = aoi, scale = 0.0003)

ds

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (12, 4))
plt.tight_layout(w_pad = 2)
ds.ndvi_before_mean.plot(ax = ax[0], x = 'lon', y = 'lat', robust = True, cmap = 'RdYlGn')
ds.ndvi_after_mean.plot(ax = ax[1], x = 'lon', y = 'lat', robust = True, cmap = 'RdYlGn')
ax[0].set_title('NDVI Before Fires in Early 2019')
ax[1].set_title('NDVI After Fires in Early 2019')

plt.savefig('ndvi_before_after.png', dpi = 360, bbox_inches = 'tight')


In [ ]:
ds.ndvi_change_mean.plot(x = 'lon', y = 'lat', cmap = 'Spectral', robust = True)
plt.savefig('ndvi_change_spectral.png', dpi = 360, bbox_inches = 'tight')

In [ ]:
thr = ds.ndvi_change_mean > 0.15
thr.plot(x = 'lon', y = 'lat', cmap = 'Greys')
plt.savefig('ndvi_change_greys.png', dpi = 360, bbox_inches = 'tight')